Imports and Preprocessing


In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Concatenate, Multiply, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

# Directory paths
data_dir = r'C:/Users/rishi/OneDrive/Desktop/cnnmodel'

# Image dimensions
img_width, img_height = 224, 224
batch_size = 32

# Data Augmentation and Rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 8012 images belonging to 2 classes.
Found 2003 images belonging to 2 classes.


2. Attention Mechanism

In [9]:
def spatial_attention(input_feature):
    avg_pool = tf.reduce_mean(input_feature, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(input_feature, axis=-1, keepdims=True)
    concat = tf.concat([avg_pool, max_pool], axis=-1)
    attention = Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')(concat)
    return Multiply()([input_feature, attention])


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Concatenate, Multiply, Conv2D
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.models import Model

def spatial_attention(input_feature):
    input_feature = tf.convert_to_tensor(input_feature)  # Ensure input is a Tensor
    avg_pool = tf.reduce_mean(input_feature, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(input_feature, axis=-1, keepdims=True)
    concat = tf.concat([avg_pool, max_pool], axis=-1)
    attention = Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')(concat)
    return Multiply()([input_feature, attention])


# Input layer
input_shape = (224, 224, 3)  # Image dimensions
input_tensor = Input(shape=input_shape)

# EfficientNetB0 and ResNet50 backbones
efficient_net = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor, pooling=None)
resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor, pooling=None)

# Ensure same feature map sizes
efficient_output = efficient_net.output  # Shape: (None, 7, 7, 1280) for EfficientNetB0
resnet_output = resnet.output  # Shape: (None, 7, 7, 2048) for ResNet50

# Apply attention mechanism to both outputs
efficient_attention = spatial_attention(efficient_output)
resnet_attention = spatial_attention(resnet_output)

# Merge the outputs
merged = Concatenate()([efficient_attention, resnet_attention])

# Global Average Pooling and Dropout
x = GlobalAveragePooling2D()(merged)
x = Dropout(0.5)(x)

# Output layer (binary classification)
output = Dense(1, activation='sigmoid')(x)

# Build and compile the model
model = Model(inputs=input_tensor, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
# Callbacks for saving the best model and early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_skin_cancer_model.h5', monitor='val_accuracy', save_best_only=True)


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping, checkpoint],
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size
)


In [ ]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_acc}')


In [ ]:
# Generate predictions
y_pred = model.predict(validation_generator)
y_pred = np.where(y_pred > 0.5, 1, 0)

# Confusion matrix and classification report
print(confusion_matrix(validation_generator.classes, y_pred))
print(classification_report(validation_generator.classes, y_pred))


In [ ]:
from tensorflow.keras.preprocessing import image

# Load and preprocess a new image for prediction
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.  # Normalize to [0, 1] range

    # Predict
    prediction = model.predict(img_array)
    
    # Interpret prediction
    if prediction > 0.5:
        print("Prediction: Positive for skin cancer")
    else:
        print("Prediction: Negative for skin cancer")

# Example: Upload an image and predict
image_path = r'C:/Users/rishi/OneDrive/Desktop/cnnmodel/HAM10000_images_part_1/ISIC_0024306.jpg'
predict_image(image_path)
